In [16]:
import torch
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [12]:
model_name = "bert-base-uncased"
max_length = 512

In [20]:
#Load Dataset
def read_20newsgroups(test_size=0.2):
  dataset = fetch_20newsgroups(subset="all", shuffle=True, remove=("headers", "footers", "quotes"))
  documents = dataset.data
  labels = dataset.target
  return train_test_split(documents, labels, test_size=test_size), dataset.target_names
  
(train, test, train_target, test_target), target_names = read_20newsgroups()

In [21]:
#Load the Tokenizer
tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)

#Tokanize train and test data
train = tokenizer(train, truncation=True, padding=True, max_length=max_length)
test = tokenizer(test, truncation=True, padding=True, max_length=max_length)

In [22]:
#Encode the data to torch
class NewsGroupsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)
        
train_dataset = NewsGroupsDataset(train, train_target)
test_dataset = NewsGroupsDataset(test, test_target)

In [ ]:
#Convert model to cuda GPU (Running this on colap to use GPU)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(target_names)).to("cuda")

In [24]:
#Accuracy Function needed in training
def Accuracy(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }

In [ ]:
#Define training Arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=200,               # log & save weights each logging_steps
    evaluation_strategy="steps",     # evaluate each `logging_steps`
)
#Define the trainer
trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,          # evaluation dataset
    compute_metrics=Accuracy,     # the callback that computes metrics of interest
)

In [ ]:
#Train the model on the new data (Training done on colap)
trainer.train()
#Check the accuracy
trainer.evaluate()

#To save the file after training
model_path = "Fine-Tune(BERT)"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

In [39]:
#Load the saved model
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('Fine-Tune(BERT)', do_lower_case=True, do_basic_tokenize=True)
model = BertForSequenceClassification.from_pretrained('Fine-Tune(BERT)')


In [40]:
def get_prediction(text):
    #Tokenize the input
    inputs = tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt")
    #Inference to the model
    outputs = model(**inputs)
    #Get output probabilty
    probs = outputs[0].softmax(1)
    #Get the actual label name
    return target_names[probs.argmax()]

In [46]:
#Testing the model
text = """
A black hole is a place in space where gravity pulls so much that even light can not get out. 
The gravity is so strong because matter has been squeezed into a tiny space. This can happen when a star is dying.
Because no light can get out, people can't see black holes. 
They are invisible. Space telescopes with special tools can help find black holes. 
The special tools can see how stars that are very close to black holes act differently than other stars.
"""
text2 = """
The first thing is first. 
If you purchase a Macbook, you should not encounter performance issues that will prevent you from learning to code efficiently.
However, in the off chance that you have to deal with a slow computer, you will need to make some adjustments. 
Having too many background apps running in the background is one of the most common causes. 
The same can be said about a lack of drive storage. 
For that, it helps if you uninstall xcode and other unnecessary applications, as well as temporary system junk like caches and old backups.
""" 
text3 = """
Coronavirus disease (COVID-19) is an infectious disease caused by a newly discovered coronavirus.
Most people infected with the COVID-19 virus will experience mild to moderate respiratory illness and recover without requiring special treatment.  
Older people, and those with underlying medical problems like cardiovascular disease, diabetes, chronic respiratory disease, and cancer are more likely to develop serious illness.
"""

print(get_prediction(text3))

sci.med
